<a href="https://colab.research.google.com/github/RyutoYoda/Instagram_API_demo1/blob/main/Instagram_API_demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests

#1：InstagramAPIを使って任意の#から投稿を取得する

In [4]:
import requests
import json

hashTagName = '（例）おしゃれカフェ'
instaBusinessId = 'ビジネスアカウントID'
accessToken = 'アクセストークン'

# InstagramAPIでハッシュタグIDを取得する関数
def getIgHashId():
    try:
        url = f'https://graph.facebook.com/v17.0/ig_hashtag_search?user_id={instaBusinessId}&q={hashTagName}&access_token={accessToken}'
        response = instagramApi(url, 'GET')

        data = response.json()
        hashTagId = data['data'][0]['id']

        return hashTagId
    except Exception as error:
        print('Instagram APIのリクエスト中にエラーが発生しました:', error)
        return None

# APIを叩く関数
def instagramApi(url, method):
    try:
        headers = {
            'Authorization': f'Bearer {accessToken}'
        }
        response = requests.request(method, url, headers=headers)
        response.raise_for_status()  # これによりHTTPエラー時に例外が発生します
        return response
    except requests.RequestException as error:
        print('Instagram APIのリクエスト中にエラーが発生しました:', error)
        return None

# 実行例
hashTagId = getIgHashId()
print("ハッシュタグID:", hashTagId)

ハッシュタグID: 17841551125094952


In [ ]:
# 最近の投稿を取得する関数
def searchTopHashTagPosts():
    # ハッシュタグIDを取得
    hashTagId = getIgHashId()

    try:
        url = f'https://graph.facebook.com/{hashTagId}/recent_media?user_id={instaBusinessId}&fields=caption,comments_count,id,like_count,media_type,media_url,permalink,timestamp,children%7Bmedia_url%7D&access_token={accessToken}&limit=35'
        response = instagramApi(url, 'GET')

        # レスポンスのステータスコードを確認
        if response.status_code != 200:
            print('Instagram APIのリクエストが失敗しました:', response.status_code)
            return None

        instagramMedia = response.json()['data']
        print(instagramMedia)  # データ出力

        return instagramMedia
    except Exception as error:
        print('Instagram APIのリクエスト中にエラーが発生しました:', error)
        return None

# 実行例#ハッシュタグ
instagramMedia = searchTopHashTagPosts()

#２：取得したデータをBigQueryに落とす。

In [ ]:
import pandas as pd
from google.colab import auth
from google.cloud import bigquery

In [ ]:
auth.authenticate_user()

In [ ]:
def flatten_children_column(instagramMedia):
    for item in instagramMedia:
        if "children" in item and isinstance(item["children"], dict) and "media_url" in item["children"]:
            item["children_media_url"] = item["children"]["media_url"]
            del item["children"]
    return instagramMedia


# データの変換
flattened_data = flatten_children_column(instagramMedia)
df = convert_to_dataframe(flattened_data)

# 2. 正しいプロジェクトIDを指定してBigQueryクライアントを作成
project_id = "プロジェクトID"
client = bigquery.Client(project=project_id)

# 3. DataFrameをBigQueryにアップロード
def upload_to_bigquery(df, dataset_name, table_name):
    dataset_ref = client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.write_disposition = "WRITE_TRUNCATE"  # 既存のテーブルを上書き
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # ジョブの完了を待機

    print(f"Loaded {df.shape[0]} rows into {dataset_name}:{table_name}.")

# データのアップロード
upload_to_bigquery(df, "データセット名", "テーブル名")

##SQLを実行しBigQueryから再度データを取得する

In [ ]:
def get_data_from_bigquery(dataset_name, table_name):
    # 1. BigQueryクライアントの初期化
    project_id = "プロジェクトID"
    client = bigquery.Client(project=project_id)

    # 2. SQLクエリの実行
    query = f"SELECT * FROM `{project_id}.{dataset_name}.{table_name}`"
    query_job = client.query(query)

    # 3. 結果をPandasのDataFrameに変換
    data = query_job.to_dataframe()

    return data

# 使用例
data = get_data_from_bigquery("データセット名", "テーブル名")
print(data.head())